In [71]:
! pip install kubeflow-metadata

     |████████████████████████████████| 4.9 MB 4.8 MB/s 
  Created wheel for kubeflow-metadata: filename=kubeflow_metadata-0.3.1-py3-none-any.whl size=11471 sha256=251568903deb5344f1b5a7523e6ba45c5544409a53cb88ac704c9dadebf1aa0e
  Stored in directory: /home/pmccarthy/.cache/pip/wheels/9b/39/1d/2c572e2fb7c629465195f44a605082fce313715ba9574993a0
  Created wheel for retrying: filename=retrying-1.3.3-py3-none-any.whl size=11429 sha256=a43750d45642799b66874d3e55db47f5b1846ef56e8dcac02dcd1018b285495d
  Stored in directory: /home/pmccarthy/.cache/pip/wheels/f9/8d/8d/f6af3f7f9eea3553bc2fe6d53e4b287dad18b06a861ac56ddf
Successfully built kubeflow-metadata retrying
  Attempting uninstall: ml-metadata
    Found existing installation: ml-metadata 0.23.0
    Uninstalling ml-metadata-0.23.0:
      Successfully uninstalled ml-metadata-0.23.0
ERROR: After October 2020 you may experience errors when installing or updating packages. This is because pip will change the way that it resolves dependency conf

In [82]:
import tensorflow_model_analysis as tfma
import tfx
from tfx.orchestration.kubeflow import kubeflow_dag_runner, kubeflow_metadata_adapter

from kubeflow.metadata import metadata

import pandas as pd
from io import StringIO
import subprocess
import yaml


In [26]:
pod_listing = subprocess.run(["kubectl","get","pods","--selector=app=mysql","-o","yaml"],capture_output=True).stdout

pod_listing_dict = yaml.safe_load(pod_listing)

In [37]:
meta_pod = subprocess.run(["kubectl","describe","services","metadata-grpc-service"],capture_output=True).stdout

In [70]:
endpoint_addr = [x for x in meta_pod.decode('utf-8').split('\n') if 'Endpoints' in x][0].replace('Endpoints:','').strip()

In [73]:
METADATA_STORE_HOST, METADATA_STORE_PORT = endpoint_addr.split(':')

In [74]:
ws1 = metadata.Workspace(store=metadata.Store(grpc_host=METADATA_STORE_HOST, grpc_port=int(METADATA_STORE_PORT)),
name='workspace_1',description='a workspace for testing',labels={"n1":"v1"})

In [79]:
eval_artifacts = ws1.store.get_artifacts_by_type('ModelEvaluation')

In [83]:
eval_results = tfma.load_eval_results([x.uri for x in eval_artifacts])

Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


In [84]:
eval_results

&lt;tensorflow_model_analysis.view.view_types.EvalResults at 0x7f4cb117b790&gt;

In [85]:
def trim_location(result):
    location_str = result._asdict()['model_location']
    return "_".join(location_str.split('/')[3:5])

def process_metrics(result):
    metric_list = result._asdict()['slicing_metrics']
    metric_dict = metric_list[0][1]['']['']
    return pd.DataFrame.from_dict(metric_dict)


result_df = pd.concat({
    trim_location(x):process_metrics(x) 
    for x in eval_results.get_results()})
result_df

,,binary_accuracy,binary_accuracy_diff,sparse_categorical_accuracy
tfx_pipeline_output_pjm-pipeline-20200909a,doubleValue,0.711934,NaN,NaN
,doubleValue,0.920047,0.208113,NaN
tfx_iris_edb0c03f-edb1-4620-90d9-c58f95eefc89,doubleValue,NaN,NaN,0.513514
